In [3]:
import requests
from bs4 import BeautifulSoup
import csv

def getSnow():
    url = "https://www.onthesnow.com/colorado/skireport"
    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")

    tags = soup.find_all("td", class_="styles_commonCell__wB0xC styles_commonRowCell__8t9yH")

    mtnInfo = []
    snowInfo = []
    baseInfo = []
    trailsInfo = []
    liftsInfo = []

    # get mountain name and last update time
    for tag in tags[::5]:
        pointer = 0
        text = tag.text
        while pointer < len(text):
            if text[pointer].isnumeric():
                mountain = text[:pointer].strip()
                update_time = text[pointer:].strip()
                break
            pointer += 1
        mtnInfo.append((mountain, update_time))



    # get snow and base depth info
    for snow in tags[1::5]:
        snowInfo.append(snow.text.strip())

    for base in tags[2::5]:
        baseInfo.append(base.text.strip())

    for trail in tags[3::5]:
        text = trail.text.strip()
        if "/" in text:
            try:
                open_trails, total_trails = text.split("/")
                open_trails = int(open_trails)
                total_trails = int(total_trails)
                percent_open = f"{(total_trails / open_trails) * 100:.0f}%"  # Convert to percentage
            except ValueError:
                percent_open = text  # Keep original if conversion fails
        else:
            percent_open = text  # Already a percentage
        trailsInfo.append(percent_open)


    for lift in tags[4::5]:
        liftsInfo.append(lift.text.strip())
    # print name, update time, snow depth, and open trails
    filename = "ski_report.csv"
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Mountain", "Last Update", "24-Hour Snow", "Base Depth", "Open Trails", "Open Lifts"])
        
        for i in range(min(len(mtnInfo), len(snowInfo), len(baseInfo), len(trailsInfo), len(liftsInfo))):
            writer.writerow([mtnInfo[i][0], mtnInfo[i][1], snowInfo[i], baseInfo[i], trailsInfo[i], liftsInfo[i]])

getSnow()
